# Explore the data

We will lated come back with the test data. So just work on df_ed and df_hh
GOAL: get a good prediction as quickly as possible
Don'e do the "Hh  poverty score continuation' for now

In [5]:
import pandas as pd
import os, sys
import matplotlib.pyplot as plt


In [6]:
# Additional covariate called family size. to USE more data. 
# How to use more data by groupping poverty_scores on household?
# within same household
# add a psu_hh uui as a VARIABLE to tell the model "predict something similar for members of the same family". It would be difficult, there is a lot.
# stacking? what is stacking?

# Root preprocessing:

In [303]:
ed = pd.read_csv('ed_train_raw.csv') # education DataFrame
hh = pd.read_csv('hh_train_raw.csv') # household DataFrame



In [305]:
### Prepare Ed data
def prepare_df(df, suffix:str):
    uids = df['psu'].astype(str) + "_"  + df['hh'].astype(str) + "_" + df['idcode'].astype(str) 
    df = df.drop(columns=['psu', 'hh', 'idcode'])
    df.columns = [suffix + "_" + col.capitalize() for col in df.columns]
    df.insert(0, 'uid', uids)
    return df

ed = prepare_df(ed, 'ED')
hh = prepare_df(hh, 'HH')

# Remove absoluteley useless variates

In [307]:
hh = hh.drop(columns=['HH_Hhid', 'HH_Q04', 'HH_Q08', 'HH_Q12', 'HH_Q18'])
columns_to_keep = [
    'uid',
    'ED_Q01', 'ED_Q02', 'ED_Q03', 'ED_Q04', 'ED_Q05', 'ED_Q06', 'ED_Q07', 'ED_Q08', 'ED_Q09', 'ED_Q10', 'ED_Q11', 
    'ED_Q14', 'ED_Q15', 'ED_Q16', 'ED_Q17', 'ED_Q18',
    'ED_Q19',
    'ED_Q23',
    'ED_Q26', 'ED_Q27', 'ED_Q28', 'ED_Q29',
    'ED_Q41', 
]

ed = ed[columns_to_keep]


# Filter data based on uids that have a poverty score assigned to them.

### This step is ONLY for TRAIN data. 

In [309]:
poverty = pd.read_csv('poverty_train_raw.csv') # poverty/answers

### compute poverty scores
for i in range(1,11):
    col = 'subjective_poverty_'+ str(i)
    poverty.loc[poverty[col]==1, 'poverty_score'] = i

poverty['uid'] = poverty['psu_hh_idcode']
poverty = poverty[['uid', 'poverty_score']]


ed = ed[ed['uid'].isin(poverty['uid'])]
ed = ed.merge(poverty[['uid', 'poverty_score']], on='uid', how='left')

hh = hh[hh['uid'].isin(poverty['uid'])]
hh = hh.merge(poverty[['uid', 'poverty_score']], on='uid', how='left')




In [243]:
# # quite a lot of data wasted... nothing we can do.... right?
# print(train_ed.shape)
# print(train_hh.shape)
# print(poverty.shape)

# # Explore test data:
# print(test_ed.shape)
# print(test_hh.shape)
# assert(test_hh[test_hh['uid'].isin(test_ed['uid'])].shape[0] == 1334)
# ## No issues here. just use them as they are

In [14]:
# df_hh.to_csv('hh_train_transform.csv', index=False)
# df_ed.to_csv('ed_train_transform.csv', index=False)
# df_pov_scores.to_csv('scores_train_transform.csv', index=False)


# Rename variates for interpretability

In [311]:
# ED variates
ed['read'] = ed['ED_Q01'] 
ed['write'] = ed['ED_Q02'] 
ed['attended_school'] = ed['ED_Q03'] 

# Standarize
ed['highest_school_lvl'] = ed['ED_Q04'] 
ed['highest_school_lvl_grade'] = ed['ED_Q05'] 
ed['highest_diploma'] = ed['ED_Q06'] 

ed['preschool'] = ed['ED_Q07'] 

# Merge
ed['now_enrolled'] = ed['ED_Q08'] 
ed['now_attending'] = ed['ED_Q09'] # 1 yes, 2 No
ed['past_enrolled'] = ed['ED_Q14'] # 1 yes, 2 No
ed['past_attending'] = ed['ED_Q15'] # 1 yes, 2 No

# Merge
ed['now_not_attend_reason'] = ed['ED_Q10']
ed['now_not_enroll_reason'] = ed['ED_Q11']
ed['past_not_attend_reason'] = ed['ED_Q16'] 
ed['past_not_enroll_reason'] = ed['ED_Q17']

ed['finish_school_age'] = ed['ED_Q18'] 

ed['younger_19'] = ed['ED_Q19'] 

ed['public_private_school'] = ed['ED_Q23'] 

ed['school_transportation_time'] = ed['ED_Q26'].apply(lambda hrs : hrs * 60) +  ed['ED_Q27'] # it works
ed['school_transportation_vehicle'] = ed['ED_Q28'] 
ed['school_transportation_cost'] = ed['ED_Q29'] 

ed['school_expenses'] = ed['ED_Q41'] 


In [313]:
new_cols = [
    'uid',
    'read', 'write','attended_school','highest_school_lvl','highest_school_lvl_grade','highest_diploma',
    'preschool','now_enrolled','now_attending','past_enrolled', 'past_attending','now_not_attend_reason',
    'now_not_enroll_reason','past_not_attend_reason','past_not_enroll_reason','finish_school_age',
    'younger_19','public_private_school','school_transportation_time','school_transportation_vehicle',
    'school_transportation_cost','school_expenses',
    'poverty_score'
]


ed = ed[new_cols]


In [317]:
ed.head()

,uid,read,write,attended_school,highest_school_lvl,highest_school_lvl_grade,highest_diploma,preschool,now_enrolled,now_attending,...,past_not_attend_reason,past_not_enroll_reason,finish_school_age,younger_19,public_private_school,school_transportation_time,school_transportation_vehicle,school_transportation_cost,school_expenses,poverty_score
0,441_2_3,1,1,1,2.0,3.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,40.0,1.0,NaN,140000.0,4.0
1,647_7_1,1,1,1,2.0,2.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,15.0,1.0,NaN,75000.0,4.0
2,756_4_1,1,1,1,2.0,3.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,6.0,1.0,NaN,100000.0,6.0
3,25_4_3,1,1,1,6.0,1.0,3.0,2.0,1.0,1.0,...,NaN,NaN,NaN,NaN,3.0,20.0,1.0,NaN,730000.0,4.0
4,132_6_3,1,1,1,2.0,3.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,30.0,2.0,NaN,170000.0,3.0


In [321]:
# HH variates
hh.columns
['uid', 
 'HH_Q02', 
 'HH_Q03', 
 'HH_Q05y', 
 'HH_Q05m', 
 'HH_Q06', 
 'HH_Q07',
 'HH_Q09', 
 'HH_Q10', 
 'HH_Q11', 
 'HH_Q13', 
 'HH_Q14', 
 'HH_Q15', 
 'HH_Q16',
 'HH_Q17', 
 'HH_Q19', 
 'HH_Q20',
 'HH_Q21', 
 'HH_Q22', 
 'poverty_score'
]


Index(['uid', 'HH_Q02', 'HH_Q03', 'HH_Q05y', 'HH_Q05m', 'HH_Q06', 'HH_Q07',
       'HH_Q09', 'HH_Q10', 'HH_Q11', 'HH_Q13', 'HH_Q14', 'HH_Q15', 'HH_Q16',
       'HH_Q17', 'HH_Q19', 'HH_Q20', 'HH_Q21', 'HH_Q22', 'poverty_score'],
      dtype='object')

# Transform ed data

#### TURN all these silly 2's INTO 0's, will simplify the model.


# Transform hh data

# Group #1 

Claim: Those who answer No = 2 for ED_Q3 are done with the ED survey. 


#### Group 1 only uses ED 1,2




In [77]:
# get group_1 train sets
group1_train = train_ed['ED_Q03'].eq(2)
ed_group1_train = train_ed[group1_train]

ed_group1_train = ed_group1_train[['uid', 'ED_Q01', 'ED_Q02']]
hh_group1_train = train_hh[train_hh['uid'].isin(ed_group1_train['uid'])]


In [79]:
# get group1 test sets
group1_test = test_ed['ED_Q03'].eq(2)
ed_group1_test = test_ed[group1_test]

ed_group1_test = ed_group1_test[['uid', 'ED_Q01', 'ED_Q02']]
hh_group1_test = test_hh[test_hh['uid'].isin(ed_group1_test['uid'])]


In [86]:
# Save them to /group1
ed_group1_train.to_csv('./group1/ed_train_g1.csv', index=False)
hh_group1_train.to_csv('./group1/hh_train_g1.csv', index=False)
ed_group1_test.to_csv('./group1/ed_test_g1.csv', index=False)
hh_group1_test.to_csv('./group1/hh_test_g1.csv', index=False)

# Group 2

The current students

they use all the variates in edu

In [173]:
buffer = train_ed.loc[~group1_train]
# Try to literally look for those who have completed school (Option 13 in either 11 or 17)
split = buffer['ED_Q11'].eq(13) | buffer['ED_Q17'].eq(13)
buffer[split].shape

(3594, 23)

In [179]:
# Those who graduated recently?
subsplit = split & buffer['ED_Q11'].ne(buffer['ED_Q17'])  #
buffer[subsplit]

,uid,ED_Q01,ED_Q02,ED_Q03,ED_Q04,ED_Q05,ED_Q06,ED_Q07,ED_Q08,ED_Q09,...,ED_Q15,ED_Q16,ED_Q17,ED_Q19,ED_Q23,ED_Q26,ED_Q27,ED_Q28,ED_Q29,ED_Q41
16,767_6_7,1,1,1,2.0,3.0,3.0,0.0,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,0.0,40.0,1.0,NaN,90000.0
26,486_7_2,1,1,1,6.0,3.0,7.0,3.0,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,0.0,40.0,1.0,NaN,2468000.0
31,747_3_3,1,1,1,8.0,2.0,10.0,3.0,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,0.0,30.0,5.0,100000.0,880000.0
34,175_8_3,1,1,1,8.0,1.0,7.0,2.0,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,0.0,10.0,1.0,NaN,760000.0
209,462_7_1,1,1,1,1.0,8.0,2.0,1.0,2.0,NaN,...,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,517_7_1,1,1,1,2.0,4.0,3.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5185,448_1_1,1,2,1,1.0,4.0,1.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5187,575_5_1,3,3,1,1.0,4.0,1.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5249,666_6_1,1,1,1,6.0,4.0,9.0,0.0,2.0,NaN,...,NaN,NaN,14.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
# Those who completed school last year
subsplit = buffer['ED_Q17'].eq(13)
buffer[subsplit]


,uid,ED_Q01,ED_Q02,ED_Q03,ED_Q04,ED_Q05,ED_Q06,ED_Q07,ED_Q08,ED_Q09,...,ED_Q15,ED_Q16,ED_Q17,ED_Q19,ED_Q23,ED_Q26,ED_Q27,ED_Q28,ED_Q29,ED_Q41
198,179_7_2,1,1,1,2.0,3.0,3.0,1.0,2.0,NaN,...,NaN,NaN,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
199,833_8_2,1,1,1,5.0,4.0,6.0,0.0,2.0,NaN,...,NaN,NaN,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1691,631_4_2,1,1,1,2.0,4.0,3.0,1.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1692,698_3_2,1,1,1,2.0,4.0,3.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1693,109_4_1,1,1,1,6.0,3.0,7.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5328,829_4_4,1,1,1,6.0,3.0,7.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5329,574_5_1,1,1,1,1.0,4.0,1.0,1.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5330,618_2_1,2,3,1,1.0,4.0,1.0,1.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5331,155_4_1,1,1,1,1.0,8.0,2.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
# Try to literally look for those who have completed school: 
ed_g2_train = buffer[split]   ## THAT"S MY GROuP 2 SON

# roman factor.... hmmmmm 
ed_g2_train

,uid,ED_Q01,ED_Q02,ED_Q03,ED_Q04,ED_Q05,ED_Q06,ED_Q07,ED_Q08,ED_Q09,...,ED_Q15,ED_Q16,ED_Q17,ED_Q19,ED_Q23,ED_Q26,ED_Q27,ED_Q28,ED_Q29,ED_Q41
16,767_6_7,1,1,1,2.0,3.0,3.0,0.0,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,0.0,40.0,1.0,NaN,90000.0
26,486_7_2,1,1,1,6.0,3.0,7.0,3.0,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,0.0,40.0,1.0,NaN,2468000.0
31,747_3_3,1,1,1,8.0,2.0,10.0,3.0,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,0.0,30.0,5.0,100000.0,880000.0
34,175_8_3,1,1,1,8.0,1.0,7.0,2.0,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,0.0,10.0,1.0,NaN,760000.0
198,179_7_2,1,1,1,2.0,3.0,3.0,1.0,2.0,NaN,...,NaN,NaN,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5328,829_4_4,1,1,1,6.0,3.0,7.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5329,574_5_1,1,1,1,1.0,4.0,1.0,1.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5330,618_2_1,2,3,1,1.0,4.0,1.0,1.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5331,155_4_1,1,1,1,1.0,8.0,2.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


# G3

#### Those who have completed school

Should not use certain ed questions in their prediction... the ones meant for students doi! determine which ones these are

In [157]:
ed_g3_train = buffer[~split]
ed_g3_train

,uid,ED_Q01,ED_Q02,ED_Q03,ED_Q04,ED_Q05,ED_Q06,ED_Q07,ED_Q08,ED_Q09,...,ED_Q15,ED_Q16,ED_Q17,ED_Q19,ED_Q23,ED_Q26,ED_Q27,ED_Q28,ED_Q29,ED_Q41
0,441_2_3,1,1,1,2.0,3.0,3.0,0.0,1.0,1.0,...,1.0,NaN,NaN,NaN,1.0,0.0,40.0,1.0,NaN,140000.0
1,647_7_1,1,1,1,2.0,2.0,2.0,0.0,1.0,1.0,...,1.0,NaN,NaN,NaN,1.0,0.0,15.0,1.0,NaN,75000.0
2,756_4_1,1,1,1,2.0,3.0,3.0,0.0,1.0,1.0,...,1.0,NaN,NaN,NaN,1.0,0.0,6.0,1.0,NaN,100000.0
3,25_4_3,1,1,1,6.0,1.0,3.0,2.0,1.0,1.0,...,1.0,NaN,NaN,NaN,3.0,0.0,20.0,1.0,NaN,730000.0
4,132_6_3,1,1,1,2.0,3.0,3.0,0.0,1.0,1.0,...,1.0,NaN,NaN,NaN,1.0,0.0,30.0,2.0,NaN,170000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5312,799_3_5,1,1,1,1.0,8.0,2.0,3.0,2.0,NaN,...,NaN,NaN,12.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5313,215_4_1,1,1,1,1.0,8.0,2.0,0.0,2.0,NaN,...,NaN,NaN,12.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5314,785_2_2,1,1,1,1.0,8.0,2.0,0.0,2.0,NaN,...,NaN,NaN,12.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5315,377_7_2,1,1,1,1.0,8.0,2.0,0.0,2.0,NaN,...,NaN,NaN,12.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# data = group2_temp[['ED_Q26', 'ED_Q27']]
# non_trivial = ~data['ED_Q26'].isna()
# data[non_trivial]


In [29]:
 # There might be a group 3 which splits from group2 at Q14? everything after 20 is 0?
group3 = group2_temp['ED_Q14'].is(2)

SyntaxError: invalid syntax (1387834729.py, line 2)

# Households

before you even split into groups, try your hand at usign households to "guess" more people's poverty score and increase the amount of data used

In [ ]:
poverty['fid'] = poverty['uid'].apply(lambda uid : '_'.join([uid.split('_')[0], uid.split('_')[1]]) ) 


In [ ]:
# The first division is by "primary sampling unit." It's like the first sample. 
# Members from the same family are in the same sample. So psh_hh is basically the family id.
# idcode is then the id of the individual..

# maybe you could average family responses into the observation with the poverty score... if poverty scores only has about one person from each family and there are more in the full data.


psu1 = train_ed[train_ed['ED_Psu']==1]
print(psu1.shape)
family_1_12 = psu1[psu1['ED_Hh']== 12]
family_1_12

In [ ]:
cond = poverty['fid'].eq("1_12")
poverty[cond]
# we could, for instance, set the poverty_scores of all three members of the family equal to each other. 
# HOWEVER, the questions is: how rich do YOU THINK YOU are? a kid with no allowance may think he is starving. 
# Even so, I think the variable: How rich does your family think you are? is good for training. 

In [ ]:
# No repeated fid's in the poverty_df. 
# So no two members of the family have an opinion on how poor/rich they are. 
# but if you are a kid and your dad says youre poor, you probably think you're poor??? there's so many vars
frequencies = poverty['fid'].value_counts()
# frequencies

In [ ]:
# are there any of the fid's in the test data also in the poverty scores? probably not, it would be super nice.
test_hh['fid'] = test_hh['uid'].apply(lambda uid : '_'.join([uid.split('_')[0], uid.split('_')[1]]) ) 
poverty[poverty['uid'].isin(test_hh['fid'])] # Nope

In [ ]:
# why are there some entries with the SAME idcodes and HOUSEHOLD to each other!????
train_ed[train_ed['hh']==10] ### psu_hh .... first houses into groups psu is first level division.
# could we continuate? psu_hh is family
hh10 = df_pov_scores[df_pov_scores['hh']==10]
hh10['poverty_score'].describe()

In [ ]:
plt.hist(hh10['poverty_score'], bins=5, color='blue', edgecolor='black', alpha=0.7)